In [1]:
!pip install firebase
## needed
!pip install langdetect

In [2]:
#[1]
## All the imports are here ##
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output , HTML
from IPython.display import display
from firebase import firebase
import json
import matplotlib.pyplot as plt
from datetime import datetime
import os
from collections import Counter
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from langdetect import detect
import calendar
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#[2]
#add styles to the elemetes (use exampleWidgit.addClass("classNamee") and use the class name to style in css)
def loadStyles():
    # Display CSS
  display(HTML('''<style>
    .widget-box {
      display: flex;
      background-color: #0D0C27;
      border: 3px dashed #00bfff;
      padding: 20px;
      border-radius: 30px;
      color: #FFB23E;
      height:450px;

      width:auto;
    }
    .widget-label {
      color: #FFB23E;
    }


    .custom-upload {
      background-color: #0D0C27;
      border: 2px dashed #00bfff;
      padding: 20px;
      text-align: center;
      border-radius: 10px;
      color: #FFB23E;
    }


    .button {
      background-color: #ff9933;
      color: #000000;
      border: none;
      border-radius: 20px;
      padding: 20px;
      cursor: pointer;
      margin: 10px;
      width:  322px;
      height: 200px;
      font-size: 20px;
    }
    .button:hover {
      background-color: #ffcc66;
    }
 .back-button {
            background-color: transparent;
            color: white;
            border: 2px solid #debf37;
            border-radius: 20px;
            padding: 20px;
            margin: 10px;
            width: 300px;
            height: 200px;
            font-size: 20px;
            position: fixed;
            bottom: 10px;
            right: 10px;
        }
        .back-button:hover {
            background-color: rgba(0, 0, 255, 0.1); /* Light blue background on hover */
        }

        .back-button-inprogressdisplay{
          height: 50px;
            position:none;

        }

        .save-button {
        position: fixed;
        bottom: 10px;
        Left: 10px;
        }




    .upload-button {
      background-color: #a6a18b;
      color: white;
      border: dashed black;
      border-radius: 20px;
      cursor: pointer;
      margin-top: 10px;
      center: center;
      width: 300px;
      height: 50px;
    }
    .upload-button:hover {
      background-color: #ffcc66;
    }
    .show_graph_button{

    }

    .widget-label {
      color: #FFB23E !important;
    }
    .widget-checkbox .widget-label {
      color: #FFB23E !important;
    }
    .title {
      font-size: 36px;
      font-weight: bold;
      text-align: center;
      color: white;
    }
    .warning {
      font-size: 10px;
      font-weight: bold;
      color: red;
    }
  </style>'''))



In [4]:
#[3]
#connect to firebase
firebase_url = 'https://learning911-p-default-rtdb.europe-west1.firebasedatabase.app/'

FBconn = firebase.FirebaseApplication(firebase_url, None)
#get data from database
def fetch_json_data():
    result = FBconn.get(firebase_url, '/data')
    if result:
      last_key = list(result.keys())[-1] if result else None
      last_entry = result[last_key] if last_key else None #get the last database uploaded
    if last_entry is None:
        print("No data found in Firebase.")
        return {}
    else:
        return last_entry

In [5]:
#[4]
def update_items(category_dropdown, item_dropdown, df):
    selected_category = category_dropdown.value
    filtered_items = df[df['Category'] == selected_category]['Item']
    item_dropdown.options = filtered_items.unique()

In [6]:
#[5]
#hamza window
def graph1(event):
  #The user isnt use the filtered before
  global filtered_df
  if filtered_df.empty:
    filtered_df=df

  if 'Time' not in df.columns:
      raise ValueError("'Time' column is missing in the data")
  filtered_df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
  plot_data = {}
  for user in filtered_df['User'].unique():
      user_df = filtered_df[filtered_df['User'] == user]
      user_df = user_df.sort_values(by='Time')

      start_time = None
      total_time = pd.Timedelta(0)
      operation_count = 0

      for _, row in user_df.iterrows():
          operation_count += 1

          if row['Description'] == "Open document":
              start_time = row['Time']
          elif row['Description'] == "Close document" and start_time is not None:
              total_time += row['Time'] - start_time
              start_time = None

      total_hours = total_time.total_seconds() / 3600

      if total_hours > 0:
          plot_data[user] = {
              'operations_per_hour': operation_count / total_hours,
              'total_operations': operation_count,
              'total_hours': total_hours
          }

  for user, data in plot_data.items():
      print(f"Student: {user}")
      print(f"  Total Operations: {data['total_operations']}")
      print(f"  Total Hours: {data['total_hours']:.2f}")
      print(f"  Operations per Hour: {data['operations_per_hour']:.2f}")

  plot_df = pd.DataFrame.from_dict(plot_data, orient='index').reset_index()

  if len(plot_df.columns) == 4:
      plot_df.columns = ['User', 'Operations per Hour', 'Total Operations', 'Total Hours']
  else:
      print("Error: Plot DataFrame does not have the expected number of columns.")

  colors = plt.cm.get_cmap('tab10', len(plot_df['User']))

  plt.figure(figsize=(10, 6))
  bars = plt.bar(plot_df['User'], plot_df['Operations per Hour'], color=[colors(i) for i in range(len(plot_df))])

  plt.xlabel('Student')
  plt.ylabel('Operations per Hour')
  plt.title('Operations per Hour for Each Student')
  plt.xticks(rotation=45)
  plt.tight_layout()

  plt.show()


In [7]:
#[6]
def open_display4(event):
  title = widgets.HTML('<h1 class="title">Operations per Hour graph</h1>')
  show_graph_button = newButton('Show Graph','button')
  show_graph_button.on_click(graph1)
  show_graph_button.add_class('show_graph_button')
  back_button =getBackToMainMenuButton()
  myDisplay('V',title,show_graph_button,back_button)

In [8]:
#[7]
#Basel window
#project progress graph
#the graph gives an how the project has progress in a time period
def getDateAndActions():
  #array for dates ,array for number of actions done in a single day
  global dateArray
  global actionPerDate
  filteredData=filtered_df
  localdateArray=[]
  localactionPerDate=[]
  if(len(localdateArray)>0 or len(localactionPerDate)>0):
    localactionPerDate.clear()
    localdateArray.clear()
  if 'Time' not in filteredData.columns:
    print("error, not time column found")
  #convert Time column to datetime to extract date easily
  filteredData['Time'] = pd.to_datetime(filteredData['Time'])
  #iterate over each row
  for index,row in filteredData.iterrows():
    #get the date only
    Date = row['Time'].date()
    # if date is fouund add 1 to localactionPerDate in the respective place
    if Date in localdateArray:
      i=localdateArray.index(Date)
      localactionPerDate[i]=localactionPerDate[i]+1
    #if date not found then add it to dateArray and add 1 (first action) in actionPerDate
    else:
      localdateArray.append(Date)
      localactionPerDate.append(1)
  dateArray=localdateArray
  actionPerDate=localactionPerDate


#get months only from date
def extract_months(dateArray):
  months = []
  for date in dateArray:
    flag=0
    month = date.month
    #if month not in months add it
    for i in range(len(months)):
      if(months[i]==month):
        flag=1
    #month not found add month
    if(flag==0):
      months.append(month)
  return months

#get years only from date
def extract_years(dateArray):
  years = []
  for date in dateArray:
    flag=0
    year = date.year
    #if year not in years add it
    for i in range(len(years)):
      if(years[i]==year):
        flag=1
    #year not found add year
    if(flag==0):
      years.append(year)
  return years

#get days for specfic month
def getDaysInMonth(month):
    #get the number of days in the specified month
    days_in_month = calendar.monthrange(2000, int(month))[1]
    #create a list of days from 1 to the number of days in the month
    days = list(range(1, days_in_month + 1))
    return days

# Function to update days dropdown based on selected month and year
def update_starting_days(change=None):
    if change['new'] is not None:
        selected_month = int(change['new'])
        days = getDaysInMonth(selected_month)
        StartingDay.options = [None] + days
    else:
        StartingDay.options=[None]

# Function to update days dropdown based on selected month and year
def update_finishing_days(change=None):
    if change['new'] is not None:
        selected_month = int(change['new'])
        days = getDaysInMonth(selected_month)
        FinishingDay.options = [None] + days
    else:
        FinishingDay.options=[None]

# Function to build date range and cumulative actions array
def build_date_range(start_date, end_date, date_array, action_array):
    date_range = pd.date_range(start=start_date, end=end_date)
    cumulative_actions = []
    sum_actions = 0
    for date in date_range:
        if date.date() in date_array:
            index = date_array.index(date.date())
            sum_actions += action_array[index]
        cumulative_actions.append(sum_actions)
    return date_range, cumulative_actions

# Function to update graph based on selected starting and finishing dates
def update_graph(change=None):
    plt.clf
    if StartingYear.value is not None and StartingMonth.value is not None and StartingDay.value is not None and \
       FinishingYear.value is not None and FinishingMonth.value is not None and FinishingDay.value is not None:
        start_date = pd.Timestamp(year=int(StartingYear.value), month=int(StartingMonth.value), day=int(StartingDay.value))
        end_date = pd.Timestamp(year=int(FinishingYear.value), month=int(FinishingMonth.value), day=int(FinishingDay.value))
        if(start_date>=end_date):
          print("The starting date must be before the finishing date")
          return
        date_range, cumulative_actions = build_date_range(start_date, end_date, dateArray, actionPerDate)
        print("The total amount of actions done in this period of time is ",cumulative_actions[len(cumulative_actions)-1])
        plt.figure(figsize=(10, 6))
        plt.plot(date_range, cumulative_actions, marker='o', linestyle='-', color='b')
        plt.title('Project Progress Over Time')
        plt.xlabel('Date')
        plt.ylabel('Cumulative Actions')
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    else:
        print("Please select starting date and finishing date")


def on_operations_per_student_click(b):
    graph1()


def on_showGraph_click(b):
  update_graph()

In [9]:
import ipywidgets as widgets

def create_date_dropdowns(label):
    Month = widgets.Dropdown(
        options=[None] + list(extract_months(dateArray)),
        description=f'{label} Month:',
        disabled=False
    )
    Year = widgets.Dropdown(
        options=[None] + list(extract_years(dateArray)),
        description=f'{label} Year:',
        disabled=False
    )
    Day = widgets.Dropdown(
        options=[None],
        description=f'{label} Day:',
        disabled=False
    )

    def update_days(change):
        if change['new'] is not None:
            Day.options = [None] + getDaysInMonth(change['new'])
        else:
            Day.options = [None]

    Month.observe(update_days, names='value')
    return Day, Month, Year

def Show_Project_progress_Form():
    global StartingDay, StartingMonth, StartingYear, FinishingDay, FinishingMonth, FinishingYear

    getDateAndActions()

    # Create dropdowns for starting and finishing dates
    StartingDay, StartingMonth, StartingYear = create_date_dropdowns("Starting")
    FinishingDay, FinishingMonth, FinishingYear = create_date_dropdowns("Finishing")

    # Button to show graph
    showGraph = newButton('Show Graph', 'button')
    showGraph.on_click(on_showGraph_click)

    # UI components
    chooseStartDate = widgets.HTML(value="<b style='color: #ffffff;'>Choose starting date</b>")
    staetDate = widgets.HBox([StartingYear, StartingMonth, StartingDay])

    chooseFinishDate = widgets.HTML(value="<b style='color: #ffffff;'>Choose finishing date</b>")
    finishDate = widgets.HBox([FinishingYear, FinishingMonth, FinishingDay])

    # Back button
    back_button = getBackToMainMenuButton()
    back_button.add_class('back-button-inprogressdisplay')

    # Display the form
    myDisplay('V', chooseStartDate, staetDate, chooseFinishDate, finishDate, showGraph, back_button)


In [10]:
#[9]
def newButton(description,className):
  b=widgets.Button(description=description)
  b.add_class(className)
  return b

In [11]:
#[10]
def create_upload_button(fileType,multiple=False):
  upload_button = widgets.FileUpload(
    accept=fileType,
    multiple=multiple,  # Do not allow multiple files
    description='Browse JSON file to upload',
    layout=widgets.Layout(width='300px')
    )
  return upload_button

In [12]:
#[11]
global mainDisplayV
mainDisplayV = widgets.VBox()
global mainDisplayH
mainDisplayH = widgets.HBox()

def myDisplay(layout,*args):
    global mainDisplay
    if layout == 'H':
      mainDisplayH.children=args
      mainDisplayV.children=[]
      mainDisplay = mainDisplayH
    elif layout == 'V':
      mainDisplayV.children=args
      mainDisplayH.children=[]
      mainDisplay = mainDisplayV
    else:

        mainDisplay = widgets.VBox(args)
    clear_output()
    loadStyles()
    display(mainDisplay)

In [13]:
#[12]
#display to show progress graph of the filtired students
def open_display3(event):
  Show_Project_progress_Form()

In [14]:
#[13]
#make global varible for filtered data (type dataframe)
def save_filter_json( document_name, user_name,event):
    global filtered_df
    # Filter based on the 'Document' and 'User' columns
    if document_name is None and user_name is None:
        filtered_df = df
    elif document_name is not None and user_name is None:
      filtered_df  = df[ (df['Document'] == document_name)]
    elif document_name is None and user_name is not None:
      filtered_df  = df[df['User'] == user_name]
    elif document_name is not None and user_name is not None:
      filtered_df  = df[(df['Document'] == document_name) & (df['User'] == user_name)]

In [15]:
#[14]
#extract csv file for the filtered data
def create_excel(event):
        filtered_df.to_csv('filtered_data.csv', index=False)
        print("Filtered data saved successfully as 'filtered_data.csv'.")

In [16]:
#[15]
#get user names from the last data uploaded to the database
def getUserNames():
  userNames = set(df['User'].dropna().unique()) #extract the user names
  return list(userNames)

In [17]:
#[16]
#get project names from the last data uploaded to the database
def getProjectsNames():
  ProjectNames = set(df['Document'].dropna().unique()) #extract the project names
  return list(ProjectNames)

In [18]:
#[17]
#filter display
def open_display2(event):
  title = widgets.HTML('<h1 class="title">Data Filtiring</h1>')
  warning=widgets.HTML('<h6 class="warning">TO display project progress graph please choose user= none</h6>')
  # Dropdown widgets
  #project list drop down
  project_dropdown = widgets.Dropdown(
      options=[None] +getProjectsNames() ,
      description='ProjectName:',
      disabled=False,
      layout=widgets.Layout(width='300px'),
      style={'description_width': 'initial'}
  )
  project_dropdown.add_class('widget-label')
  #user list drop down
  user_dropdown = widgets.Dropdown(
      options=[None]+ getUserNames(),
      description='User:',
      disabled=False,
      layout=widgets.Layout(width='300px'),
      style={'description_width': 'initial'}
  )
  user_dropdown.add_class('widget-label')

  save_button = widgets.Button(description='Save Filtered Data')
  save_button.on_click(lambda event: save_filter_json(project_dropdown.value, user_dropdown.value,event))

  create_excel_button = widgets.Button(description='Create Excel')
  create_excel_button.on_click(create_excel)

  back_button =getBackToMainMenuButton()
  myDisplay('V',title,warning,project_dropdown,user_dropdown,save_button,create_excel_button,back_button)

In [19]:
def open_display5(event):
    title = widgets.HTML('<h1 class="title">Create Index</h1>')
    title1 = widgets.HTML('<h3 class="title">The list of elements are:</h3>')
    back_button =getBackToMainMenuButton()
    # Get word frequencies from the DataFrame
    word_frequencies = count_word_frequencies()
    # Create a Textarea widget to display the word frequencies
    text_area = widgets.Textarea(
        value="\n".join(word_frequencies),
        description='Word Frequencies:',
        disabled=True,
        layout=widgets.Layout(width='600px', height='300px')
    )

    # Display the widgets
    myDisplay('V',title,title1,text_area,back_button)

In [20]:
#[18]
#save the uploaded json file to project and upload to firebase
def onSaveButton(upload,event):
  # File upload
  if upload.value:
    # Get the uploaded file content
    uploaded_file = list(upload.value.values())[0]
    content = uploaded_file['content'].decode('utf-8')
    # Load JSON content
    uploaded_json = json.loads(content)
    print("JSON file uploaded successfully!")
  else:
    print("No file uploaded.")
  # File Save to Database

    print("Save to Firebase button clicked")
  if uploaded_json:
        FBconn = firebase.FirebaseApplication(firebase_url, None)
        FBconn.post('/data', uploaded_json)
        print("Saving to Firebase...")
        # Save the JSON content to a file (as an example)
        with open('uploaded_file.json', 'w') as f:
            json.dump(uploaded_json, f)
        print("File saved successfully!")
  else:
        print("No JSON data to save.")


In [21]:
#[19]
#Upload JSON file display
def open_display1(event):
  # File upload widget
  title = widgets.HTML('<h1 class="title">Upload JSON file to database</h1>')
  upload = create_upload_button('.json')
  upload.add_class('upload-button')
  save_button = newButton('Save','button')
  save_button.add_class('save-button')
  save_button.on_click(lambda event: onSaveButton(upload, event))
  back_button =getBackToMainMenuButton()
  myDisplay('V',title,upload,save_button,back_button)

In [22]:
#[20]
#backButon
def getBackToMainMenuButton():
  back_button =newButton('Back','button')
  back_button.add_class('back-button')
  back_button.on_click(show_menu)
  return back_button

In [23]:
#[21]
#Main Menu - This is The first Page
def show_menu(event):
  title = widgets.HTML('<h1 class="title">Badger App</h1>')
  button1=newButton('Upload','button')
  button2=newButton('filter','button')
  button3=newButton('Progress','button')
  button4=newButton('Operations per Hour'  ,'button')
  button5=newButton('Create index'  ,'button')
  button1.on_click(open_display1)
  button2.on_click(open_display2)
  button3.on_click(open_display3)
  button4.on_click(open_display4)
  button5.on_click(open_display5)

  myDisplay('v',title,button1,button2,button3,button4,button5)


In [24]:
import pandas as pd
import json
from collections import Counter
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk

# Download NLTK data if you haven't already
nltk.download('punkt')

# Initialize the DataFrame variables
df = pd.DataFrame()  # Replace with your actual data
filtered_df = pd.DataFrame()

def count_word_frequencies():
    global df
    global filtered_df

    # Check if the data was filtered
    if filtered_df.empty:
        filtered_df = df

    # Initialize the stemmer
    stemmer = PorterStemmer()

    # Convert DataFrame to a single string
    text = " ".join(filtered_df.astype(str).apply(lambda row: ' '.join(row), axis=1))

    # Tokenize the text into words
    words = word_tokenize(text.lower())

    # Filter out non-alphabetic words
    words = [word for word in words if word.isalpha()]

    # Apply stemming to each word
    stemmed_words = [stemmer.stem(word) for word in words]

    # Count the frequency of each word
    word_counts = Counter(stemmed_words)

    # Convert to a list of tuples and sort by frequency
    word_list = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

    return [f"{word}: {count}" for word, count in word_list]

# Example usage:
# Make sure to assign data to df and filtered_df before calling


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
#[Last]
#main app call #
global df
global filtered_df
df=pd.DataFrame(fetch_json_data())
global uploaded_json
show_menu('event')

In [26]:
#Don't add cells under this cell

# **#Don't add cells under this cell**